In [12]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import json

In [3]:
#dowlnoad full dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [13]:
#create csv file with pre-processed data
ar_data=dataset.copy()
#ar_data=ar_data.iloc[:1000]
ar_data=ar_data[ar_data['sentiment'].isin(['normal', 'hateful'])]
ar_data.shape
print(ar_data.iloc[200])

print(ar_dataset.shape)


HITId                                                                491
tweet                  فيه واحد خنزير قاعد يعاطس على جهتي وجا شوي بلغ...
sentiment                                                        hateful
directness                                                      indirect
annotator_sentiment                                         indifference
target                                                             other
group                                                         individual
Name: 490, dtype: object


NameError: name 'ar_dataset' is not defined

In [14]:
#dataset separate by languages
arab_path = "/Users/mathey/Desktop/deeplearning/dataset/ar_dataset_600.csv"
ar_dataset = pd.read_csv(arab_path)
#print(ar_dataset.shape)
#print(ar_dataset.head())
ar_dataset = ar_dataset.sample(frac=1, random_state=42)


en_path="/Users/mathey/Desktop/deeplearning/dataset/en_dataset_600.csv"
en_dataset = pd.read_csv(en_path)
#print(en_dataset.shape)
#print(en_dataset.head())
en_dataset = en_dataset.sample(frac=1, random_state=42)


fr_path="/Users/mathey/Desktop/deeplearning/dataset/fr_dataset_600.csv"
fr_dataset = pd.read_csv(fr_path)
#print(fr_dataset.shape)
#print(fr_dataset.head())
fr_dataset = fr_dataset.sample(frac=1, random_state=42)

#create binary datasets
en_dataset_binary=en_dataset.copy()
en_dataset_binary = en_dataset_binary[en_dataset_binary['sentiment_binary'].isin([1, 3])]
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(1, 0)
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(3, 1)

ar_dataset_binary=ar_dataset.copy()
ar_dataset_binary = ar_dataset_binary[ar_dataset_binary['sentiment_binary'].isin([1, 3])]
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(1, 0)
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(3, 1)

fr_dataset_binary=fr_dataset.copy()
fr_dataset_binary = fr_dataset_binary[fr_dataset_binary['sentiment_binary'].isin([1, 3])]
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(1, 0)
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(3, 1)




FileNotFoundError: [Errno 2] No such file or directory: '/Users/mathey/Desktop/deeplearning/dataset/ar_dataset_600.csv'

In [5]:
default_prompt = "Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Text:"
default_prompt_en_hate_speech_def = "The definition of hate speech is ‘Hate speech’ is speech that attacks a person or group on the basis of attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Text:"
default_prompt_fr_hate_speech_def = "La définition du discours de haine est la suivante : le « discours de haine » est un discours qui attaque une personne ou un groupe en raison d’attributs tels que la race, la religion, l'origine ethnique, l'origine nationale, le sexe, le handicap, l'orientation sexuelle ou l'identité de genre. Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Text:"
default_prompt_ar_hate_speech_def = "تعريف خطاب الكراهية هو: 'خطاب الكراهية' هو خطاب يهاجم شخصًا أو مجموعة بناءً على سمات مثل العرق، الدين، الأصل العرقي، الأصل الوطني، الجنس، الإعاقة، التوجه الجنسي، أو الهوية الجندرية. Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Text:"
default_prompt_hate_speech_def_3_languages = "The definition of hate speech is ‘Hate speech’ is speech that attacks a person or group on the basis of attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. تعريف خطاب الكراهية هو: 'خطاب الكراهية' هو خطاب يهاجم شخصًا أو مجموعة بناءً على سمات مثل العرق، الدين، الأصل العرقي، الأصل الوطني، الجنس، الإعاقة، التوجه الجنسي، أو الهوية الجندرية. La définition du discours de haine est la suivante : le « discours de haine » est un discours qui attaque une personne ou un groupe en raison d’attributs tels que la race, la religion, l'origine ethnique, l'origine nationale, le sexe, le handicap, l'orientation sexuelle ou l'identité de genre. Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Text:"
few_shot_en_examples = "Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Example 1:  Text: 'i dont think im getting my baby them white 9 he has two white j and nikes not even touched'. Answer: No. Example 2: Text: 'we cannot continue calling ourselves feminists if the right of all womxn arent addressed yes to a sexual offences public list but will a trans lesbian bisexual and queer womxn be able to enter their information on the reporting sheet gender forum'. Answer: No. Example 3: Text: 'tariq nasheed beating the fuck out of that drum beloved lmao somebody put a biscuit in that niggas mouth already'. Answer: No. Example 3: Text: 'tariq nasheed beating the fuck out of that drum beloved lmao somebody put a biscuit in that niggas mouth already'. Answer: No. Example 4: Text: 'cotton thigh highs to avoid contact with any gay chemicals'. Answer: No. Example 5: Text: 'fellas be careful she texting other niggas'. Answer: No. Example 6: Text: 'laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate'. Answer: Yes. Example 7: Text: 'lol what a stupid kike'. Answer: Yes. Example 8: Text: 'this sounds like a kike national socialist'. Answer: Yes. Example 9: Text: 'just another dumb nigger'. Answer: Yes. Example 10: Text: 'they are going to figure out a way to kike bitcoin'. Answer: Yes. Text:"
few_shot_en_fr_ar_examples = "Is the following text hateful or not? Just answer in Yes or No. Don't provide explanations. Example 1:  Text: 'i dont think im getting my baby them white 9 he has two white j and nikes not even touched'. Answer: No. Example 2: Text: 'we cannot continue calling ourselves feminists if the right of all womxn arent addressed yes to a sexual offences public list but will a trans lesbian bisexual and queer womxn be able to enter their information on the reporting sheet gender forum'. Answer: No. Example 3: Text: 'just another dumb nigger'. Answer: Yes. Example 4: Text: 'they are going to figure out a way to kike bitcoin'. Answer: Yes. Example 5: Text: 'يارب انصرنا #افريقيا_يا_اهلي'. Example 6: Text: 'الصور العارية واجسام البنات ومقاطع منحرفة ، ستشهد عليها الاصابع يوم القيامة ، كفاكم استهانة قالجنة اغلى'. Answer: No. Example 7: 'بس يا شارب بول البعير يا عبد ال سلول'. Answer: Yes. Example 8: Text: 'هههعععععع اي علوم ههههه بول البعير'. Answer: Yes. Example 9: Text: 'ohhh la violence je meuuurs'. Answer: No. Example 10: Text: 'pour ou contre l'avortement ?'. Answer: No. Example 11: Text: 'tu nous ramène une maladie de jérusalem?'. Anwer: Yes. Example 12: Text: 'le secours catholique ardent promoteur de l'ensauvagement et de l'islamisation de l'europe ! son catholicisme dévo…' Answer: Yes. Text:"
few_shot_and_definition_prompt="Is the following text hateful or not? Definition of hate speech: Hate speech refers to any form of expression (written, spoken, or otherwise communicated) that promotes hatred, violence, discrimination, or hostility towards individuals or groups based on attributes such as race, ethnicity, religion, gender, sexual orientation, nationality, disability, or other identifiable characteristics. Hate speech can manifest in various forms, including derogatory language, slurs, threats, incitement to violence, or dehumanizing statements targeting a particular group. Example of hate speech: 'All [ethnic group] are criminals and should be deported.', 'People with [disability] are a burden on society and should be euthanized.', 'We need to exterminate all [religious group] to purify our nation.', 'Women belong in the kitchen and should not be allowed to hold positions of power.'. Guiddelines for classification: 'Yes' if the text contains language or expressions that clearly promote hatred, discrimination, violence, or hostility towards individuals or groups based on their inherent characteristics, mark it as hate speech. 'No' if the text does not exhibit such characteristics and is devoid of any explicit or implicit expressions of hate, mark it as not hate speech. Just answer in Yes or No. Don't provide explanations. Text:"
ask_for_resonning="Is the following text hateful or not? Start by stating: 'Yes' or 'No'. Then display the words that make text hatefull or not. Finally give a short explaination of why it is hate or not. Text:"


In [189]:
#other models
used_data = en_dataset_binary
prompt=default_prompt
predictions_concat = []


# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
   
    prompt = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": tweet}
    ]
   
    # LLM initialization
    llm = Ollama(model="llama3")
    response = llm.invoke(prompt)
    
    if response.strip().lower() == 'yes':
        predictions = 1
    elif response.strip().lower() == 'no':
        predictions = 0
    else:
        predictions = 0  # Default to non-hateful if the response is unexpected
   
    predictions_concat.append(predictions)


print(len(predictions_concat))


# Add the predictions to the dataset
used_data['predictions'] = predictions_concat


# Metrics
acc = accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1 = f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision = precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall = recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')


print(f"Accuracy: {acc:.3f}, F1: {F1:.3f}, Precision: {Precision:.3f}, Recall: {Recall:.3f}")


  0%|          | 0/40 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [7]:
#Code to run Mistral and prompt 9
used_data = en_dataset_binary
used_prompt=default_prompt
predictions_concat = []


# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = used_data['tweet'].iloc[i]
   
    prompt = [
        {"role": "system", "content": used_prompt},
        {"role": "user", "content": tweet}
    ]
   
    # LLM initialization
    llm = Ollama(model="mistral")
    response = llm.invoke(prompt)
    
    match = re.search(r'(Yes|No|YES|NO|yes|no)', response)

    if match:
        answer = match.group(1).lower()
        if answer == "yes":
            predictions = 1  # Assign 1 if the answer is "Yes"
        elif answer == "no":
            predictions = 0  # Assign 0 if the answer is "No"
        else:
            predictions = 0  # If the answer is neither "Yes" nor "No"
            print("Answer not found1")
    else:
        predictions = 0  # If the answer part is not found
        print("Answer not found2")
    
    
    predictions_concat.append(predictions)


print(len(predictions_concat))


# Add the predictions to the dataset
used_data['predictions'] = predictions_concat


# Metrics
acc = accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1 = f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision = precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall = recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')


print(f"Accuracy: {acc:.3f}, F1: {F1:.3f}, Precision: {Precision:.3f}, Recall: {Recall:.3f}")


 57%|█████▊    | 230/400 [30:27<24:43,  8.73s/it]  

Answer not found2


 80%|████████  | 321/400 [43:08<07:30,  5.71s/it]

In [180]:
print(predictions_concat)



[0, 0, 0]


In [6]:
print(en_dataset_binary[[ 'sentiment','sentiment_binary']])


    sentiment  sentiment_binary
110    normal                 0
419   hateful                 1
565   hateful                 1
77     normal                 0
181    normal                 0
..        ...               ...
20     normal                 0
71     normal                 0
106    normal                 0
435   hateful                 1
102    normal                 0

[400 rows x 2 columns]
